In [ ]:
from tqdm.notebook import tqdm
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
import re
import numpy as np
import ast

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 40)
pd.set_option('display.max_colwidth', None)

In [ ]:
df = pd.read_pickle("../data/events/df_events 2025-01-10_16-41-46.pkl")

In [ ]:
columns_order = ['StartDate', 'StartTime',  "EndDate", "EndTime", 'StartDateTime', "EndDateTime"]
for col in df.columns:
    if col not in columns_order:
        columns_order.append(col)


In [ ]:
# reorder columns 
df = df[columns_order]  

In [ ]:
df = df.sample(frac=1, random_state=42).reset_index(drop=True)



# preprocess address columns

## fill in StreetHouseNumber from LocationDetails

In [ ]:
flt = df.Street.isna() & df.StreetHouseNumber.isna() & (df.LocationDetails!="")
df.loc[flt, "LocationDetails"].dropna().value_counts()  

In [ ]:
df["LocationDetails_orig"]=df.LocationDetails

In [ ]:
df.LocationDetails = df.LocationDetails.str.replace("\d{5}", "", regex=True)

In [ ]:
df.LocationDetails = df.LocationDetails.str.replace("Dresden-Altstadt", "", regex=False)
df.LocationDetails = df.LocationDetails.str.replace("Dresden-Prohlis", "", regex=False)
df.LocationDetails = df.LocationDetails.str.replace("Dresden-Pillnitz ", "", regex=False)
df.LocationDetails = df.LocationDetails.str.replace("Dresden", "", regex=False)
df.LocationDetails = df.LocationDetails.str.replace("-Langebrück", "", regex=False)

df.LocationDetails = df.LocationDetails.str.replace("Freiburg im Breisgau", "", regex=False)
df.LocationDetails = df.LocationDetails.str.replace("Freiburg im breisgau", "", regex=False)
df.LocationDetails = df.LocationDetails.str.replace("Freiburg", "", regex=False)
df.LocationDetails = df.LocationDetails.str.replace("Germany", "", regex=False)
df.LocationDetails = df.LocationDetails.str.replace(",", "", regex=False)
df.LocationDetails = df.LocationDetails.str.replace("  ", " ", regex=False)
df.LocationDetails = df.LocationDetails.str.strip()

In [ ]:
flt = df.Street.isna() & df.StreetHouseNumber.isna() & (df.LocationDetails!="")
df.loc[flt, "LocationDetails"].dropna().value_counts().sort_index()

In [ ]:
flt = df.Street.isna() & df.StreetHouseNumber.isna() & (df.LocationDetails!="") 
df.loc[flt, "StreetHouseNumber"] = df.loc[flt, "LocationDetails"]

### from StreetHouseNumber to Street and HouseNumber

In [ ]:
df["Street_orig"] = df["Street"]

In [ ]:
df["StreetHouseNumber_orig"] = df["StreetHouseNumber"]

In [ ]:
df.StreetHouseNumber = df.StreetHouseNumber.str.replace("Dresden, Alemania", "", regex=False)
df.StreetHouseNumber = df.StreetHouseNumber.str.replace("Dresden, Germany", "", regex=False)
df.StreetHouseNumber = df.StreetHouseNumber.str.replace("Freiburg im Breisgau", "", regex=False)
df.StreetHouseNumber = df.StreetHouseNumber.str.replace("\d{5}", "", regex=True)
df.StreetHouseNumber = df.StreetHouseNumber.str.replace(",", "", regex=False)
df.StreetHouseNumber = df.StreetHouseNumber.str.strip()

In [ ]:
df.StreetHouseNumber = df.StreetHouseNumber.str.replace("\d{5}", "", regex=True)

In [ ]:
df.StreetHouseNumber = df.StreetHouseNumber.str.replace("Dresden", "", regex=False)
df.StreetHouseNumber = df.StreetHouseNumber.str.replace("Freiburg", "", regex=False)
df.StreetHouseNumber = df.StreetHouseNumber.str.replace("  ", " ", regex=False)

In [ ]:
#df.StreetHouseNumber = df.StreetHouseNumber.astype(str)

In [ ]:
df.StreetHouseNumber.value_counts()

In [ ]:
df.StreetHouseNumber.fillna("", inplace=True)

In [ ]:
df.StreetHouseNumber = df.StreetHouseNumber.astype(str)

In [ ]:

#split_street_hn= df.StreetHouseNumber.str.extract(r'^(.*\D)\s*(\d+)$')
split_street_hn= df.StreetHouseNumber.str.extract(r'^(.*\D)\s+(\d[\d\-\/]*)$')

In [ ]:
split_street_hn.columns = ["Street_split", "HouseNumber_split"]
split_street_hn

In [ ]:
split_street_hn.Street_split.value_counts().sort_index()

In [ ]:
split_street_hn.HouseNumber_split.value_counts().sort_index()

In [ ]:
df.loc[139]

In [ ]:
split_street_hn.loc[139]

In [ ]:
flt = (df.Street.isna()) & (~split_street_hn.Street_split.isna())
df.loc[flt, ["Street", "StreetHouseNumber"]]
values = split_street_hn.loc[flt, "Street_split"]
values
df.loc[flt, "Street"] = values

In [ ]:
flt = (df.HouseNumber.isna())  & (~split_street_hn.HouseNumber_split.isna())
df.loc[flt, ["HouseNumber", "StreetHouseNumber"]]
values = split_street_hn.loc[flt, "HouseNumber_split"]
values
df.loc[flt, "HouseNumber"] = values


### standardize street

In [ ]:
df.City.value_counts()


In [ ]:
df.City = df.City.str.replace("im Breisgau", "")
df.City = df.City.str.replace("im breisgau", "")
df.City = df.City.str.replace("b Dresden", "")
df.City = df.City.str.strip()

In [ ]:
df.rename(columns = {
    "StartTimeIsAssumed": "StartTimeIsUnknown",
}, inplace=True)

In [ ]:
df.Street = df.Street.str.replace("Dresden, Germany", "")
df.Street = df.Street.str.replace("Dresden, Alemania", "")
df.Street = df.Street.str.replace("Aufgang B", "")

In [ ]:
print(len(df[df.lat.isna()]))

In [ ]:
flt =( df.lat.isna()) & (~df.Street.isna())
print(len(df))
print(len(df[flt]))
#df[flt]

In [ ]:
#df[df.Street.str.contains("Theaterplatz", na=False)]

In [ ]:
df.Street = df.Street.str.lower()

In [ ]:
df.Street = df.Street.str.replace("str.", "straße")

In [ ]:
df.Street = df.Street.str.replace("strasse", "straße")

In [ ]:
df.Street = df.Street.str.replace(",", "")

In [ ]:
df.Street.value_counts().sort_index()

In [ ]:
#flt = df.HouseNumber.str.contains("[a-zA-Z]", na=	False)
#df.loc[flt, "HouseNumber"].value_counts()

In [ ]:
df.head()

In [ ]:
#flt = (~df.Street.isna()) & (df.HouseNumber.isna())  & (df.StreetHouseNumber.isna())
#print(len(df[flt]))
#df[flt]

In [ ]:
df.Street = df.Street.str.replace("\d{5}", "", regex=True)
df.HouseNumber = df.HouseNumber.str.replace("\d{5}", "", regex=True)

In [ ]:
for pat in [ "(\d+)(\s)+[a-z]", "(\d+)[a-z]", "(\d+)\-(\d+)", "(\d+)"]:
    flt = (df.Street.str.contains(pat, na=False, regex=True)) & (df.HouseNumber.isna())
    values = df.loc[flt].Street.str.extract(pat, expand=False)
    df.loc[flt, "HouseNumber"] = values
    df.Street = df.Street.str.replace(pat, "", regex=True)


In [ ]:
df.Street.value_counts().sort_index()

In [ ]:
df.Street = df.Street.str.replace("dresden", "")

In [ ]:
df.HouseNumber.value_counts()

In [ ]:
df.Street = df.Street.str.replace("  ", " ", regex=False)
df.Street = df.Street.str.replace("/", "", regex=False)
df.Street = df.Street.str.replace(" - ", "", regex=False)
df.Street = df.Street.str.replace("pl.", "platz", regex=False)


In [ ]:
flt = df.Street.str.contains("altbriesnitz", na=False, regex=True)
df.loc[flt, ["Street", "Street_orig", "HouseNumber"]].drop_duplicates()

In [ ]:
df.Street = df.Street.str.strip()

In [ ]:
sorted(df.Street.dropna().unique().tolist())

In [ ]:
df.Street=df.Street.replace(" straße", "straße", regex=False)

In [ ]:
flt = df.Street == "am"
df[flt]

## get precision of coordinates

In [ ]:
#df_tmp = df.dropna(sbset=["lat"]).copy()

In [ ]:
df["lat_str"] = df.lat.astype(str)

In [ ]:
df["lat_decimal"] = df.lat_str.str.extract(r"\.(\d+)", expand=False)

In [ ]:
#df[["lat","lat_decimal", "origin_id", "Source"]]

In [ ]:
#df_tmp["lat_decimal"] = df_tmp.lat.apply(lambda x: x % 1)

In [ ]:
df["lat_decimal_precision"]=df.lat_decimal.str.len()

In [ ]:
df.dropna(subset="lat")["lat_decimal_precision"].value_counts().sort_index()

In [ ]:
df.loc[df.lat_decimal_precision==5, ["lat", "lon", "lat_decimal", "Source"]]

In [ ]:
df.loc[df.lat_decimal_precision<6, ["Source", "lat_decimal_precision"]].value_counts()  

In [ ]:
df.loc[df.lat_decimal_precision<6, ["Source"]].value_counts()  

In [ ]:
df.Source.value_counts()

In [ ]:
df.lat_decimal_precision = df.lat_decimal_precision.fillna(0)

# interpolate coordinates from own values 

In [ ]:
flt = df.lat == 0
print(len(df[flt])) # 12
df.loc[flt, "lat"] = np.nan

In [ ]:
flt = df.lon == 0
print(len(df[flt])) # 12
df.loc[flt, "lon"] = np.nan

In [ ]:
flt = df.lon.isna()
df.loc[flt, "lon"].iloc[0]

In [ ]:
print(len(df[df.lat.isna()]))

In [ ]:
pd.Series([np.nan, 3, 4]).mean()

In [ ]:
df_city_street_hn_group = df[['City', 'Street', 'HouseNumber']].dropna(subset="Street")

In [ ]:
df_city_street_hn_group

In [ ]:
df[df.lat.isna()]

In [ ]:
for i, row in df_city_street_hn_group.iterrows():
    city = row.City
    street = row.Street
    house_number = row.HouseNumber

    flt_source = (df.City == city) & (df.Street == street) & (df.HouseNumber == house_number) & (df.lat_decimal_precision>=6)
    if len(df[flt_source]) == 0:
        continue
    lat_avg = df.loc[flt_source, "lat"].mean()
    lon_avg = df.loc[flt_source, "lon"].mean()

    flt_target = (df.City == city) & (df.Street == street) & (df.HouseNumber == house_number) & (df.lat_decimal_precision<6)
    if len(df[flt_target]) == 0:
        continue
    #print(df.loc[flt_source])
    #print()
    #print(df.loc[flt_target])
    

    df.loc[flt_target, "lat"] = lat_avg
    df.loc[flt_target, "lon"] = lon_avg

    print(city, street, house_number, lat_avg, lon_avg)
    

    

# regenerate precision of coordinates

In [ ]:
df["lat_str"] = df.lat.astype(str)

In [ ]:
df["lat_decimal"] = df.lat_str.str.extract(r"\.(\d+)", expand=False)

In [ ]:
df["lat_decimal_precision"]=df.lat_decimal.str.len()

In [ ]:
df["lat_decimal_precision"].value_counts().sort_index()

In [ ]:
df.loc[df.lat_decimal_precision==5, ["lat", "lon", "lat_decimal", "Source", "StreetHouseNumber", "Street", "HouseNumber"]]

In [ ]:
df.loc[df.lat_decimal_precision<6, ["Source", "lat_decimal_precision"]].value_counts()  

In [ ]:
df.loc[df.lat_decimal_precision<6, ["Source"]].value_counts()  

In [ ]:
#df["new_lat"] = df.groupby(['City', 'Street', 'HouseNumber'])['lat'].transform(lambda x: x.fillna(x.mean()))
#df["new_lon"] = df.groupby(['City', 'Street', 'HouseNumber'])['lon'].transform(lambda x: x.fillna(x.mean()))

In [ ]:
#df.loc[30, ['City', 'Street', 'HouseNumber', "lat", "new_lat"]]

In [ ]:
#flt =( df.lat.isna()) & (~df.new_lat.isna())
#print(len(df.loc[flt]))
#df.loc[flt, ["lat", "new_lat"]]

In [ ]:
#flt =( df.lon.isna()) & (~df.new_lon.isna())
#print(len(df.loc[flt]))
#df.loc[flt, ["lon", "new_lon"]]

In [ ]:
print(len(df[df.lat.isna()]))

In [ ]:
print(len(df[df.lon.isna()]))

In [ ]:
#df_empty_lat =df[df.lat.isna()]

In [ ]:
df.head()

# Save

In [ ]:
time = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
df.to_pickle(f"../data/events/df_events {time}.pkl")
df.to_csv(f"../data/events/df_events {time}.csv")
